In [1]:
import numpy as np
import pandas as pd
import datetime    

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.utils.validation import column_or_1d

In [2]:
df_city = pd.read_csv("city_attributes.csv")
df_humidity = pd.read_csv("humidity.csv")
df_pressure = pd.read_csv("pressure.csv")
df_temper = pd.read_csv("temperature.csv")
df_descript = pd.read_csv("weather_description.csv")
df_direction = pd.read_csv("wind_direction.csv")
df_speed = pd.read_csv("wind_speed.csv")

In [3]:
df_temper.describe(include='all')

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,45253,44458.000000,45252.000000,44460.000000,45250.000000,45250.000000,45252.000000,45252.000000,45250.000000,45252.000000,...,45250.000000,44460.000000,45250.000000,45250.000000,44455.000000,44460.000000,44461.000000,44455.000000,44456.000000,44460.000000
unique,45253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-11-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,283.862654,284.992929,288.155821,284.409626,290.846116,290.215044,292.424887,295.493358,285.617856,...,285.374168,285.400406,280.343010,283.779823,291.521986,294.512307,296.497276,295.266398,294.094803,293.184253
std,NaN,6.640131,7.452438,5.332862,6.547986,6.460823,5.889992,10.829522,9.916743,9.853484,...,10.242377,10.220932,11.953626,9.802499,7.821815,6.676412,8.852984,6.324566,6.304118,7.093583
min,NaN,245.150000,262.370000,272.300000,263.780000,266.503667,265.783333,260.561333,266.059000,255.042333,...,250.390000,250.774000,243.300000,249.540000,272.179000,271.049000,271.150000,271.150000,268.682000,272.974000
25%,NaN,279.160000,279.850000,284.670000,279.830000,286.380000,286.254750,283.920000,287.680000,277.970000,...,277.350636,277.370000,271.971750,276.090000,285.366623,289.450000,289.734000,290.467333,289.881833,287.524279
50%,NaN,283.450000,284.320000,287.610000,283.940000,290.530000,290.118750,292.027486,295.586667,286.120000,...,285.927583,285.870000,281.109000,284.133250,290.932667,294.900000,296.150000,294.820000,294.150000,292.996000
75%,NaN,288.600785,289.451750,291.015167,288.530000,295.080000,294.107542,300.835000,303.050000,292.835643,...,293.796000,293.760000,290.369583,291.620000,297.270000,299.800000,303.150000,299.660000,298.930000,299.150000


In [4]:
def group_by_days(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).mean()
    return df

def group_by_days_descript(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.fillna('no data')
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).agg(lambda x: pd.Series.mode(x)[0])
    return df

In [5]:
df_humidity = group_by_days(df_humidity)
df_pressure = group_by_days(df_pressure)
df_temper = group_by_days(df_temper)
df_descript = group_by_days_descript(df_descript)
df_direction = group_by_days(df_direction)
df_speed = group_by_days(df_speed)

In [6]:
encoder = LabelEncoder()
df_descript= df_descript.apply(encoder.fit_transform)

In [7]:
def create_dataframe_for_city(city_name):
    df =  pd.concat([
        df_humidity[city_name], df_pressure[city_name], df_temper[city_name], df_descript[city_name], df_direction[city_name], df_speed[city_name]
    ], axis=1)
    df.columns = ['humidity', 'pressure', 'temperature', 'description', 'wind_direction','wind_speed' ]
    return df

In [8]:
df_Portland  =  create_dataframe_for_city("Portland")

In [9]:
df_Portland

,humidity,pressure,temperature,description,wind_direction,wind_speed
date,,,,,,
2012-10-01,78.727273,1024.000000,282.118197,11,57.727273,0.000000
2012-10-02,65.833333,1023.583333,286.137728,11,214.041667,1.291667
2012-10-03,66.208333,1021.083333,289.599792,2,228.333333,2.625000
2012-10-04,51.166667,1022.875000,286.482500,12,206.750000,4.625000
2012-10-05,40.391304,1022.916667,288.286042,12,182.250000,3.708333
...,...,...,...,...,...,...
2017-11-26,87.625000,1004.500000,281.588333,8,153.750000,4.958333
2017-11-27,92.875000,1015.625000,279.922500,8,158.083333,1.666667
2017-11-28,88.458333,1019.791667,280.085833,8,144.166667,3.041667


# Przewidywanie temperatury

In [56]:
X = df_Portland.iloc[:-2].copy()
y =  df_Portland['temperature'].iloc[4:] 

X.reset_index(inplace= True)
X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
X['month'] = pd.to_datetime(X['date']).dt.month
# X['day_sin'] = np.sin(X['day_of_year']/365 * 2 * np.pi)
# X['day_cos'] = np.cos(X['day_of_year']/365 * 2 * np.pi)
# X = X.drop(columns = ['day_of_year'])

X = X.drop('date', axis=1)
column_names = X.columns.to_list()
X= pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
X

,humidity,pressure,temperature,description,wind_direction,wind_speed,day_of_year,month,humidity_1,pressure_1,...,day_of_year_1,month_1,humidity_2,pressure_2,temperature_2,description_2,wind_direction_2,wind_speed_2,day_of_year_2,month_2
0,78.727273,1024.000000,282.118197,11,57.727273,0.000000,275,10,65.833333,1023.583333,...,276,10,66.208333,1021.083333,289.599792,2,228.333333,2.625000,277,10
1,65.833333,1023.583333,286.137728,11,214.041667,1.291667,276,10,66.208333,1021.083333,...,277,10,51.166667,1022.875000,286.482500,12,206.750000,4.625000,278,10
2,66.208333,1021.083333,289.599792,2,228.333333,2.625000,277,10,51.166667,1022.875000,...,278,10,40.391304,1022.916667,288.286042,12,182.250000,3.708333,279,10
3,51.166667,1022.875000,286.482500,12,206.750000,4.625000,278,10,40.391304,1022.916667,...,279,10,40.750000,1023.333333,288.291042,12,134.041667,3.666667,280,10
4,40.391304,1022.916667,288.286042,12,182.250000,3.708333,279,10,40.750000,1023.333333,...,280,10,36.681818,1021.000000,288.262500,12,119.166667,4.500000,281,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1878,94.083333,1016.791667,285.447917,8,120.000000,5.583333,326,11,88.250000,1013.833333,...,327,11,83.083333,1019.166667,282.543750,0,199.583333,2.791667,328,11
1879,88.250000,1013.833333,288.386667,7,174.166667,4.666667,327,11,83.083333,1019.166667,...,328,11,87.291667,1017.541667,279.293333,8,140.500000,2.500000,329,11
1880,83.083333,1019.166667,282.543750,0,199.583333,2.791667,328,11,87.291667,1017.541667,...,329,11,87.625000,1004.500000,281.588333,8,153.750000,4.958333,330,11
1881,87.291667,1017.541667,279.293333,8,140.500000,2.500000,329,11,87.625000,1004.500000,...,330,11,92.875000,1015.625000,279.922500,8,158.083333,1.666667,331,11


In [57]:
X_train, X_test, y_train, y_test = train_test_split(
           X, y, test_size = 0.3, random_state = 0,shuffle=False)

In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = ColumnTransformer(
    [("standarizer", StandardScaler(), ['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed',
                                        'humidity_1', 'pressure_1', 'temperature_1', 'wind_direction_1', 'wind_speed_1',
                                        'humidity_2', 'pressure_2', 'temperature_2', 'wind_direction_2', 'wind_speed_2'
                                        ]),
    ("one_hot_encoder", OneHotEncoder(), ['description', 'description_1', 'description_2', 'month', 'month_1', 'month_2'])],
    remainder=StandardScaler())
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [59]:
net = MLPRegressor(random_state=0, max_iter=1000, activation ='relu', learning_rate = 'adaptive', learning_rate_init = 0.001, hidden_layer_sizes=(500))

In [60]:
net.fit(X_train, y_train)

c:\Users\Filip\.pyenv\pyenv-win\versions\3.12.0\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=500, learning_rate='adaptive', max_iter=1000,
             random_state=0)

In [61]:
def abs_error(A,B):
    err = 0
    for i in range(len(A)):
        err = max(err, abs(A.iloc[i] - B[i]))
    return err

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)


In [62]:
from sklearn.metrics import mean_absolute_error
y_pred1 = net.predict(X_train)
err = abs_error(y_train, y_pred1)
print("Max absolute error")
print(err)

print("Mean absolute error")
print(mean_absolute_error(y_train, y_pred1))
margin = 2
print(f"Fraction of predictions with absolute error <= {margin}")
print(valid_in_margin(y_pred1, y_train, margin))

Max absolute error
4.862330244682596
Mean absolute error
0.7766335736269414
Fraction of predictions with absolute error <= 2
0.9271623672230652


In [63]:
from sklearn.metrics import mean_absolute_error
y_pred2 = net.predict(X_test)
err2 = abs_error(y_test, y_pred2)
print("Max absolute error")
print(err2)

print("Mean absolute error")
print(mean_absolute_error(y_test, y_pred2))
margin = 2
print(f"Fraction of predictions with absolute error <= {margin}")
print(valid_in_margin(y_pred2, y_test, margin))

Max absolute error
12.326636302053771
Mean absolute error
2.962180022666784
Fraction of predictions with absolute error <= 2
0.42300884955752216


## Przewidywanie siły wiatru

In [152]:
df_speed_2 = pd.read_csv("wind_speed.csv")
df_speed_2["date"] = pd.to_datetime(df_speed_2["datetime"]).dt.date
df_speed_2 = df_speed_2.drop(columns=['datetime'])
df_speed_2 = df_speed_2.groupby('date').apply(lambda group: (group >= 6).any())

In [153]:
X_strong_wind = df_speed_2['Portland'].apply(lambda x: 1 if x else 0)
X = df_Portland.copy()
X['strong_wind'] = X_strong_wind
X = X.iloc[:-2]
y = X_strong_wind.iloc[4:] 

X.reset_index(inplace= True)
X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
X['month'] = pd.to_datetime(X['date']).dt.month

X = X.drop('date', axis=1)
column_names = X.columns.to_list()
X= pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
X

,humidity,pressure,temperature,description,wind_direction,wind_speed,strong_wind,day_of_year,month,humidity_1,...,month_1,humidity_2,pressure_2,temperature_2,description_2,wind_direction_2,wind_speed_2,strong_wind_2,day_of_year_2,month_2
0,78.727273,1024.000000,282.118197,11,57.727273,0.000000,0,275,10,65.833333,...,10,66.208333,1021.083333,289.599792,2,228.333333,2.625000,0,277,10
1,65.833333,1023.583333,286.137728,11,214.041667,1.291667,1,276,10,66.208333,...,10,51.166667,1022.875000,286.482500,12,206.750000,4.625000,1,278,10
2,66.208333,1021.083333,289.599792,2,228.333333,2.625000,0,277,10,51.166667,...,10,40.391304,1022.916667,288.286042,12,182.250000,3.708333,1,279,10
3,51.166667,1022.875000,286.482500,12,206.750000,4.625000,1,278,10,40.391304,...,10,40.750000,1023.333333,288.291042,12,134.041667,3.666667,1,280,10
4,40.391304,1022.916667,288.286042,12,182.250000,3.708333,1,279,10,40.750000,...,10,36.681818,1021.000000,288.262500,12,119.166667,4.500000,1,281,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1878,94.083333,1016.791667,285.447917,8,120.000000,5.583333,1,326,11,88.250000,...,11,83.083333,1019.166667,282.543750,0,199.583333,2.791667,1,328,11
1879,88.250000,1013.833333,288.386667,7,174.166667,4.666667,1,327,11,83.083333,...,11,87.291667,1017.541667,279.293333,8,140.500000,2.500000,0,329,11
1880,83.083333,1019.166667,282.543750,0,199.583333,2.791667,1,328,11,87.291667,...,11,87.625000,1004.500000,281.588333,8,153.750000,4.958333,1,330,11
1881,87.291667,1017.541667,279.293333,8,140.500000,2.500000,0,329,11,87.625000,...,11,92.875000,1015.625000,279.922500,8,158.083333,1.666667,0,331,11


In [155]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct_wind = ColumnTransformer(
    [("standarizer", StandardScaler(), ['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed',
                                        'humidity_1', 'pressure_1', 'temperature_1', 'wind_direction_1', 'wind_speed_1',
                                        'humidity_2', 'pressure_2', 'temperature_2', 'wind_direction_2', 'wind_speed_2'
                                        ]),
    ("one_hot_encoder", OneHotEncoder(), ['description', 'description_1', 'description_2', 'month', 'month_1', 'month_2']),
    ('pass', 'passthrough', ['strong_wind', 'strong_wind_1', 'strong_wind_2'])],
    remainder=StandardScaler())

X_train, X_test, y_train, y_test = train_test_split(
           X, y, test_size = 0.3, random_state = 0,shuffle=False)
X_train = ct_wind.fit_transform(X_train)
X_test = ct_wind.transform(X_test)
    
net = MLPClassifier(random_state=0, max_iter=1000, activation ='relu', learning_rate = 'adaptive', learning_rate_init = 0.001, hidden_layer_sizes=1000)

In [156]:
net.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=1000, learning_rate='adaptive', max_iter=1000,
              random_state=0)

In [157]:
from sklearn.metrics import roc_auc_score
y_pred1 = net.predict_proba(X_train)[:,1]
print(roc_auc_score(y_train, y_pred1))

1.0


In [158]:
from sklearn.metrics import roc_auc_score
y_pred1 = net.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_pred1))

0.6132054794520548
